## Imports

In [ ]:

%load_ext autoreload
%autoreload 2

import os
import sys
import pickle
import pathlib
import numpy as np
import pandas as pd
import pprint as pp
import pysindy as ps
from scipy import fft
from pathlib import Path
from scipy.integrate import solve_ivp
from sklearn.preprocessing import MinMaxScaler

# Ignore matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Seed the random number generators for reproducibility
np.random.seed(100)

# Update path to include mypkg
sys.path.insert(0, str(Path(os.path.abspath('')).parent.parent.absolute()))

from src import helpers, plot_data, global_config, datasets
config = global_config.config



## Load data (and smooth it)

In [2]:
# Hyperparameters

# Initial condition parameters
n_avg = 1 # number of curves for moving average
u_true_cutoff = 150 # final index for MNIST propogated wave
u_true_start = 10

# IVP parameters
dx = None
x = None
t = None
dt = None


In [3]:

# Load the data
dataset = "MNIST"
output_dir = pathlib.Path(config.top_dir) / "fft_images" / dataset

file = output_dir / "pkl" / f"{dataset.lower()}_og_integral.pkl"
with open(file, 'rb') as file:
    d = pickle.load(file)

# Calculate moving average
u_total = helpers.moving_average(d['ints'],n=n_avg,axis=0)
u_true = u_total[u_true_start:u_true_cutoff, :]
x = np.asarray(d['r_x'])
t_total, t_true = (np.arange(0,u_total.shape[0]) + 0., np.arange(0,u_true.shape[0]) + 0.)
dt = 1.


In [4]:

# Plots
plot_data.plot_surface(
        z=u_total,x=x,y=t_total,
        xaxis_title="x", yaxis_title="time", zaxis_title="u(t,x)",
        title='PDE Input (Full Data)',
        hovertemplate='t: %{y:0.2f}<br>x: %{x:0.2f}<br> u: %{z:0.2f}<extra></extra>',
        colorscale='agsunset'
    )

plot_data.plot_surface(
        z=u_true, x=x, y=t_total,
        xaxis_title="x", yaxis_title="time", zaxis_title="u(t,x)",
        title='PDE Input (Cropped Data)',
        hovertemplate='t: %{y:0.2f}<br>x: %{x:0.2f}<br> u: %{z:0.2f}<extra></extra>',
        colorscale='agsunset'
    )

# Plot slices of u
xs = [x,x,x]
ys = [u_true[0,:], u_true[30,:], u_true[-1,:]]
labels = [f"u(x,t=0)", f"u(x,t=30)", f"u(x,t={u_true.shape[0]-1})"]
title = "Time slices of u(x,t)"

plot_data.plot_line(x=xs, y=ys, label=labels, title=title)



## Rescale

In [5]:

print(x.dtype)
x.min()
x_scaled, x_min, x_max = helpers.min_max_fit(x, 0., 1.)
t_scaled, t_min, t_max = helpers.min_max_fit(t_true, 0., 1.)
u_scaled, u_min, u_max = helpers.min_max_fit(u_true, 0., 1.)

plot_data.plot_surface(
        z=u_scaled, x=x_scaled, y=t_scaled,
        xaxis_title="x", yaxis_title="time", zaxis_title="u(t,x)",
        title='PDE Input (Cropped and Scaled Data)',
        hovertemplate='t: %{y:0.2f}<br>x: %{x:0.2f}<br> u: %{z:0.2f}<extra></extra>',
        colorscale='agsunset'
    )

x_inv = helpers.min_max_fit_inv(x_scaled, x_min, x_max, 0., 1.,)
t_inv = helpers.min_max_fit_inv(t_scaled, t_min, t_max, 0., 1.,)
u_inv = helpers.min_max_fit_inv(u_scaled, u_min, u_max, 0., 1.,)

print(t_inv[-1] - t_inv[-2])

print("||u_inv - u_true||_2", np.linalg.norm(u_inv-u_true))


int64


1.0
||u_inv - u_true||_2 3.434577253370758e-17


## Feature Library

In [6]:

u = u_scaled; x = x_scaled; t = t_scaled;
dx = x[1]-x[0]; dt = t[1]-t[0]

dummy_u = np.random.randn(x.shape[0], t.shape[0], 1)

# Define PDE library that is quadratic in u, and
# third-order in spatial derivatives of u.
if 0:
    pde_lib = ps.PDELibrary(function_library=ps.PolynomialLibrary(degree=2,include_bias=False),
                            derivative_order=3, spatial_grid=x,
                            include_bias=True, is_uniform=True)

else:
    multiindices = [[3]]
    library_functions = [lambda x: x, lambda x: x**3]
    function_names = [lambda x: f"{x}", lambda x: f"{x}^3"]
    feature_indices = [0,1,2,5]
    pde_lib = ps.PDELibrary(function_library=ps.CustomLibrary(library_functions=library_functions,function_names=function_names,include_bias=False),
                            derivative_order=3, spatial_grid=x,
                            include_bias=True, is_uniform=True, multiindices = multiindices, feature_indices=feature_indices)

dummy_pde_lib = pde_lib
dummy_pde_lib.fit([dummy_u])
feature_names = [helpers.modify_pde_sindy_out(feature) for feature in dummy_pde_lib.get_feature_names()]
print("Library:")
print(feature_names)


Library:
['1', 'u', 'u^3', 'u^3u_xxx']


## Prepare input

In [7]:

# Convert to PySINDy format
if u_scaled.shape[1] != len(t_scaled):
    u_scaled = u_scaled.T
if u_scaled.shape[1] != len(t_scaled):
    raise Exception(f"u shape is weird: {u.shape}, x shape {x.shape}, t shape {t.shape}")
u_in = np.expand_dims(u_scaled, -1)
dt = t_scaled[1] - t_scaled[0]

# Split into train and test
train_split = 1.0
u_train = u_in[:, :int(train_split*u_in.shape[1]), :]
u_test = u_in[:, int(train_split*u_in.shape[1]):, :]

print(u_scaled.shape)
print(x_scaled.shape)
print(t_scaled.shape)
print(u_train.shape)
print(u_test.shape)



(14, 140)
(14,)
(140,)
(14, 140, 1)
(14, 0, 1)


## Ensemble fit

In [8]:

ensemble_optimizer = ps.EnsembleOptimizer(
    ps.STLSQ(threshold=20, alpha=1e-5, normalize_columns=True),
    bagging=True,
    n_subset=int(0.9 * u_train.shape[0] * u_train.shape[1]),
    n_models=10000
)

model = ps.SINDy(optimizer=ensemble_optimizer, feature_names=feature_names, feature_library=pde_lib)
model.fit(u_train, t=dt)


AttributeError: 'PDELibrary' object has no attribute 'is_uniform'

AttributeError: 'PDELibrary' object has no attribute 'is_uniform'

AttributeError: 'PDELibrary' object has no attribute 'is_uniform'

In [9]:
ensemble_scores = np.asarray(ensemble_optimizer.score_list).squeeze()
ensemble_coefs = np.asarray(ensemble_optimizer.coef_list).squeeze()
mean_ensemble = np.mean(ensemble_coefs, axis=0)
std_ensemble = np.std(ensemble_coefs, axis=0)

# Collect scores for full set
full_scores = list()
for i in range(ensemble_coefs.shape[0]):
    model.optimizer.coef_ = ensemble_coefs[i][np.newaxis,:]
    full_scores.append(model.score(u_train))
full_scores = np.asarray(full_scores)

if 1:
    # Filter for 75th percentile
    percentile_75_1 = pd.DataFrame(ensemble_scores).describe().loc['75%'].item()
    percentile_75_2 = pd.DataFrame(full_scores).describe().loc['75%'].item()
    ensemble_coefs = ensemble_coefs[(ensemble_scores >= percentile_75_1) & (full_scores >= percentile_75_2), :]
    ensemble_scores = ensemble_scores[(ensemble_scores >= percentile_75_1) & (full_scores >= percentile_75_2)]
else:
    # Don't filter
    pass

print("coefs shape:", ensemble_coefs.shape)
print("num features:", len(model.get_feature_names()))
print("mean:", mean_ensemble.shape)
print("std:", std_ensemble.shape)

df = pd.DataFrame(ensemble_coefs)
df.describe()




coefs shape: (1921, 4)
num features: 4
mean: (4,)
std: (4,)


,0,1,2,3
count,1921.0,1921.0,1921.0,1921.0
mean,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0


## Make Plot

In [10]:

plot_data.plot_ensemble(feature_names, mean_ensemble, std_ensemble, title="Mean and Std of Ensemble", xaxis_title="Feature", yaxis_title="Coefficient Value", save=False)
